### Health Data

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Life Expectancy at Birth

In [2]:
url = 'https://data.cdc.gov/resource/w9j2-ggv5.json'
r = requests.get(url)
df = pd.DataFrame(r.json())
date = lambda x: pd.to_datetime(f'{x}-07-01')
df['date'] = df['year'].apply(date)
df = (df.set_index('date')
        .pivot(columns=['race', 'sex'], 
               values=['average_life_expectancy']))
df.columns = df.columns.to_series().str.join('_')
df.to_csv(data_dir / 'cdc_life_exp_raw.csv', 
          index_label='date')

In [3]:
df = pd.read_csv(data_dir / 'cdc_life_exp_raw.csv', 
                 index_col='date', parse_dates=True)
data = df.iloc[:, :3]
rn = {'average_life_expectancy_All Races_Both Sexes': 'Total', 
      'average_life_expectancy_All Races_Female': 'Women', 
      'average_life_expectancy_All Races_Male': 'Men'}

data = data.rename(rn, axis=1).loc['1989':]
data.to_csv(data_dir / 'life_exp.csv', index_label='date')
wcol = 'blue!80!black'
mcol = 'purple'
tcol = 'cyan'
grps = [('Total', tcol, None), ('Men', mcol, None), 
        ('Women', wcol, 'y')]
nodes = ('\n'.join([end_node(data[name], color, date=date, size=1.2,
                             full_year=True, colon=False) 
                    for name, color, date in grps]))
write_txt(text_dir / 'life_exp_node.txt', nodes)
url = 'https://www.cdc.gov/nchs/nvss/life-expectancy.htm'
ltdt = dtxt(data.index[-1])['year']
lttval = data.Total.iloc[-1]
prtval = data.loc['2014-07-01', ['Total']]
chtval = (lttval - prtval)
chttxt = value_text(chtval['Total'], 'increase_of', None) + ' years'
if chtval['Total'].round(1) == 0:
    chttxt = 'unchanged'
prtval2 = data.loc['1989-07-01', ['Total']]
chtval2 = (lttval - prtval2)
chttxt2 = value_text(chtval2['Total'], 'increase_of', None) + ' years'
if chtval2['Total'].round(1) == 0:
    chttxt2 = 'unchanged'
ab = ('and' if np.sign(chtval['Total']) == np.sign(chtval2['Total']) 
      else 'but')
ltmval = data.Men.iloc[-1]
prmval = data.loc['2014-07-01', 'Men']
prmval2 = data.loc['1989-07-01', 'Men']
ltwval = data.Women.iloc[-1]
prwval = data.loc['2014-07-01', 'Women']
prwval2 = data.loc['1989-07-01', 'Women']
url2 = 'https://www.cdc.gov/nchs/data/vsrr/VSRR10-508.pdf'
txt1 = ('\\textbf{{Life expectancy}} at birth summarizes '+
        'the health and mortality of a population. The '+
        'measure indicates the number of years a newborn is '+
        'expected to live if mortality rates do not change. '+
        f'Life expectancy estimates are \href{{{url}}}'+
        '{produced} by the National Center for Health '+
        'Statistics.')
write_txt(text_dir / 'life_exp.txt', txt1)
print(txt1, '\n')
txt2 = (f'In {ltdt}, US life expectancy '+
        f'at birth is {lttval:.1f} years {c_line(tcol)}, '+
        f'{chttxt} years since 2014, {ab} {chttxt2} years '+
        f'since 1989. Life expectancy for men is {ltmval:.1f} '+
        f'years in {ltdt}, compared to {prmval:.1f} years '+
        f'in 2014 and {prmval2:.1f} years in 1989 '+
        f'{c_line(mcol)}. Women born in {ltdt} are expected '+
        f'to live {ltwval:.1f} years, based on current '+
        f'mortality rates, compared to estimates of '+
        f'{prwval:.1f} years for 2014 and {prwval2:.1f} years '+
        f'for 1989 {c_line(wcol)}.\n\nFalling life expectancy '+
        'from 2014 to 2018 is generally associated with '+
        'increased overdose deaths and the opioid epidemic. '+
        'Life expectancy fell further during the COVID-19 '+
        f'pandemic, \href{{{url2}}}{{according}} to early '+
        'estimates. ')
write_txt(text_dir / 'life_exp2.txt', txt2)
print(txt2)

\textbf{{Life expectancy}} at birth summarizes the health and mortality of a population. The measure indicates the number of years a newborn is expected to live if mortality rates do not change. Life expectancy estimates are \href{https://www.cdc.gov/nchs/nvss/life-expectancy.htm}{produced} by the National Center for Health Statistics. 

In 2018, US life expectancy at birth is 78.7 years (see {\color{cyan}\textbf{---}}), a decrease of 0.2 years years since 2014, but an increase of 3.6 years years since 1989. Life expectancy for men is 76.2 years in 2018, compared to 76.5 years in 2014 and 71.7 years in 1989 (see {\color{purple}\textbf{---}}). Women born in 2018 are expected to live 81.2 years, based on current mortality rates, compared to estimates of 81.3 years for 2014 and 78.5 years for 1989 (see {\color{blue!80!black}\textbf{---}}).

Falling life expectancy from 2014 to 2018 is generally associated with increased overdose deaths and the opioid epidemic. Life expectancy fell further